# پرامپٹ انجینئرنگ کا تعارف

پرامپٹ انجینئرنگ وہ عمل ہے جس میں قدرتی زبان کی پروسیسنگ کے کاموں کے لیے پرامپٹس کو ڈیزائن اور بہتر بنایا جاتا ہے۔ اس میں درست پرامپٹس کا انتخاب، ان کے پیرامیٹرز کو ایڈجسٹ کرنا، اور ان کی کارکردگی کا جائزہ لینا شامل ہے۔ پرامپٹ انجینئرنگ این ایل پی ماڈلز میں زیادہ درستگی اور کارکردگی حاصل کرنے کے لیے نہایت اہم ہے۔ اس حصے میں، ہم اوپن اے آئی ماڈلز کو استعمال کرتے ہوئے پرامپٹ انجینئرنگ کی بنیادی باتوں کا جائزہ لیں گے۔


### مشق 1: ٹوکنائزیشن
ٹوکنائزیشن کو tiktoken کے ذریعے دریافت کریں، جو کہ OpenAI کا اوپن سورس اور تیز رفتار ٹوکنائزر ہے۔
مزید مثالوں کے لیے [OpenAI Cookbook](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_count_tokens_with_tiktoken.ipynb?WT.mc_id=academic-105485-koreyst) دیکھیں۔


In [ ]:
# EXERCISE:
# 1. Run the exercise as is first
# 2. Change the text to any prompt input you want to use & re-run to see tokens

import tiktoken

# Define the prompt you want tokenized
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

# Set the model you want encoding for
encoding = tiktoken.encoding_for_model("gpt-3.5-turbo")

# Encode the text - gives you the tokens in integer form
tokens = encoding.encode(text)
print(tokens);

# Decode the integers to see what the text versions look like
[encoding.decode_single_token_bytes(token) for token in tokens]

### مشق ۲: گیٹ ہب ماڈلز کی کلید کی سیٹنگ کی تصدیق کریں

نیچے دیا گیا کوڈ چلائیں تاکہ یہ جان سکیں کہ آپ کا گیٹ ہب ماڈلز اینڈ پوائنٹ درست طور پر سیٹ ہے یا نہیں۔ یہ کوڈ ایک سادہ پرامپٹ آزما کر اس کی تکمیل کی تصدیق کرتا ہے۔ ان پٹ `oh say can you see` پر تکمیل کچھ اس طرح ہونی چاہیے: `by the dawn's early light..`


In [2]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"

model_name = "gpt-4o"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

def get_completion(prompt, client, model_name, temperature=1.0, max_tokens=1000, top_p=1.0):
    response = client.complete(
        messages=[
            {
                "role": "system",
                "content": "You are a helpful assistant.",
            },
            {
                "role": "user",
                "content": prompt,
            },
        ],
        model=model_name,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p
    )
    return response.choices[0].message.content

## ---------- Call the helper method

### 1. Set primary content or prompt text
text = f"""
oh say can you see
"""

### 2. Use that in the prompt template below
prompt = f"""
```{text}```
"""

## 3. Run the prompt
response = get_completion(prompt, client, model_name)
print(response)


That line is the opening lyric of "The Star-Spangled Banner," the national anthem of the United States, written by Francis Scott Key. If you'd like more information or analysis, feel free to ask!


### مشق ۳: من گھڑت باتیں  
یہ جاننے کی کوشش کریں کہ جب آپ LLM سے ایسے موضوع پر جواب مانگتے ہیں جو شاید وجود ہی نہ رکھتا ہو، یا ایسے موضوعات پر جو اس کے تربیتی ڈیٹا میں شامل نہ ہوں (یعنی حالیہ معلومات)، تو کیا ہوتا ہے۔ دیکھیں کہ اگر آپ دوسرا سوال پوچھیں یا کوئی اور ماڈل استعمال کریں تو جواب کیسے بدلتا ہے۔


In [ ]:

## Set the text for simple prompt or primary content
## Prompt shows a template format with text in it - add cues, commands etc if needed
## Run the completion 
text = f"""
generate a lesson plan on the Martian War of 2076.
"""

prompt = f"""
```{text}```
"""

response = get_completion(prompt, client, model_name)
print(response)

### مشق ۴: ہدایت پر مبنی  
"text" متغیر کو بنیادی مواد کے طور پر استعمال کریں  
اور "prompt" متغیر میں اس بنیادی مواد سے متعلق ایک ہدایت فراہم کریں۔

یہاں ہم ماڈل سے کہتے ہیں کہ وہ متن کو دوسری جماعت کے طالب علم کے لیے خلاصہ کرے۔


In [4]:
# Test Example
# https://platform.openai.com/playground/p/default-summarize

## Example text
text = f"""
Jupiter is the fifth planet from the Sun and the \
largest in the Solar System. It is a gas giant with \
a mass one-thousandth that of the Sun, but two-and-a-half \
times that of all the other planets in the Solar System combined. \
Jupiter is one of the brightest objects visible to the naked eye \
in the night sky, and has been known to ancient civilizations since \
before recorded history. It is named after the Roman god Jupiter.[19] \
When viewed from Earth, Jupiter can be bright enough for its reflected \
light to cast visible shadows,[20] and is on average the third-brightest \
natural object in the night sky after the Moon and Venus.
"""

## Set the prompt
prompt = f"""
Summarize content you are provided with for a second-grade student.
```{text}```
"""

## Run the prompt
response = get_completion(prompt, client, model_name)
print(response)

Jupiter is the fifth planet from the Sun and the biggest one in our Solar System. It's made of gas and is much bigger than all the other planets put together! You can see Jupiter in the night sky because it's very bright. People have noticed it for a really long time and named it after a Roman god.


### مشق ۵: پیچیدہ پرامپٹ  
ایک درخواست آزمائیں جس میں سسٹم، صارف اور اسسٹنٹ کے پیغامات شامل ہوں  
سسٹم اسسٹنٹ کا سیاق و سباق طے کرتا ہے  
صارف اور اسسٹنٹ کے پیغامات کثیر مرحلہ گفتگو کا سیاق و سباق فراہم کرتے ہیں

نوٹ کریں کہ اسسٹنٹ کی شخصیت "طنزیہ" سسٹم کے سیاق و سباق میں مقرر کی گئی ہے۔  
کسی اور شخصیت کا سیاق و سباق استعمال کرنے کی کوشش کریں۔ یا ان پٹ/آؤٹ پٹ پیغامات کی کوئی اور ترتیب آزمائیں


In [5]:
response = client.complete(
    model=model_name,
    messages=[
        {"role": "system", "content": "You are a sarcastic assistant."},
        {"role": "user", "content": "Who won the world series in 2020?"},
        {"role": "assistant", "content": "Who do you think won? The Los Angeles Dodgers of course."},
        {"role": "user", "content": "Where was it played?"}
    ]
)
print(response.choices[0].message.content)

Oh, you mean the famous 2020 World Series that wasn’t in a regular location? That was the year they played in the glamorous Arlington, Texas, at Globe Life Field.


### مشق: اپنی بصیرت کو دریافت کریں
اوپر دیے گئے مثالیں آپ کو ایسے انداز فراہم کرتی ہیں جن کی مدد سے آپ نئے پرامپٹس بنا سکتے ہیں (سادہ، پیچیدہ، ہدایتی وغیرہ) - کوشش کریں کہ مزید مشقیں تیار کریں تاکہ آپ ان دیگر خیالات کو بھی دریافت کر سکیں جن پر ہم نے بات کی ہے جیسے مثالیں، اشارے اور دیگر۔



---

**اعلانِ دستبرداری**:  
یہ دستاویز AI ترجمہ سروس [Co-op Translator](https://github.com/Azure/co-op-translator) کے ذریعے ترجمہ کی گئی ہے۔ اگرچہ ہم درستگی کے لیے کوشاں ہیں، براہ کرم آگاہ رہیں کہ خودکار ترجمے میں غلطیاں یا عدم درستگی ہو سکتی ہے۔ اصل دستاویز کو اس کی اصل زبان میں مستند ماخذ سمجھا جانا چاہیے۔ اہم معلومات کے لیے پیشہ ور انسانی ترجمہ تجویز کیا جاتا ہے۔ اس ترجمے کے استعمال سے پیدا ہونے والی کسی بھی غلط فہمی یا غلط تشریح کی ذمہ داری ہم پر نہیں ہوگی۔
